선형 보간이 잘 이루어지지 않은 쌍과 잘 이루어지는 쌍을 가져와 보간 네트워크에 넣은 다음 학습 속도를 확인.  
후자의 상황에서 재투영오차가 더 빠르게 0에 수렴할것.  

In [1]:
import torch
import torch.nn as nn

class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1), 
            nn.BatchNorm2d(out_channels), 
            nn.SiLU(), 
            nn.Conv2d(out_channels, out_channels, 3, 1, 1), 
            nn.BatchNorm2d(out_channels)
            )
        
        self.shortcut = nn.Conv2d(in_channels, out_channels, 1) if in_channels != out_channels else nn.Identity()
        self.silu = nn.SiLU()
    def forward(self, x): return self.silu(self.block(x) + self.shortcut(x))

class EncoderSuperDeep(nn.Module):
    def __init__(self, in_channels=3, base_channels=128, latent_channels=1):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, base_channels, 3, 1, 1), 
            ResBlock(base_channels, base_channels), 
            nn.Conv2d(base_channels, base_channels*2, 3, 2, 1), 
            ResBlock(base_channels*2, base_channels*2), 
            nn.Conv2d(base_channels*2, base_channels*4, 3, 2, 1), 
            ResBlock(base_channels*4, base_channels*4), 
            nn.Conv2d(base_channels*4, base_channels*8, 3, 2, 1), 
            ResBlock(base_channels*8, base_channels*8), 
            nn.Conv2d(base_channels*8, base_channels*16, 3, 2, 1), 
            ResBlock(base_channels*16, base_channels*16), 
            nn.Conv2d(base_channels*16, 2 * latent_channels, 3, 1, 1)
            )
        
    def forward(self, x):
        x = self.encoder(x)
        mu, log_var = torch.chunk(x, 2, dim=1)
        return mu, log_var

class DecoderSuperDeep(nn.Module):
    def __init__(self, out_channels=3, base_channels=128, latent_channels=1):
        super().__init__()
        self.decoder = nn.Sequential(
            nn.Conv2d(latent_channels, base_channels*16, 3, 1, 1),
            ResBlock(base_channels*16, base_channels*16), nn.Upsample(scale_factor=2, mode='nearest'), 
            nn.Conv2d(base_channels*16, base_channels*8, 3, 1, 1), 
            ResBlock(base_channels*8, base_channels*8), nn.Upsample(scale_factor=2, mode='nearest'), 
            nn.Conv2d(base_channels*8, base_channels*4, 3, 1, 1), 
            ResBlock(base_channels*4, base_channels*4), 
            nn.Upsample(scale_factor=2, mode='nearest'), 
            nn.Conv2d(base_channels*4, base_channels*2, 3, 1, 1), 
            ResBlock(base_channels*2, base_channels*2), 
            nn.Upsample(scale_factor=2, mode='nearest'), 
            nn.Conv2d(base_channels*2, base_channels, 3, 1, 1), 
            ResBlock(base_channels, base_channels), 
            nn.Conv2d(base_channels, out_channels, 3, 1, 1), 
            nn.Tanh()
            )

    def forward(self, z): return self.decoder(z)

In [2]:
import torch
import torch.nn as nn

class InterpolationNet(nn.Module):
    """
    두 잠재 벡터와 보간 계수(alpha)를 입력받아, 
    학습된 매니폴드 위를 따라가는 새로운 잠재 벡터를 출력하는 네트워크.
    """
    def __init__(self, latent_dim=256, hidden_dim=1024):
        super().__init__()
        
        # 입력 차원: z_A(256) + z_B(256) + alpha(1) = 513
        input_dim = latent_dim * 2 + 1
        
        self.network = nn.Sequential(
            # 입력층: 입력을 받아 더 넓은 차원으로 확장하여 특징 상호작용을 학습할 공간을 제공
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim), # 각 레이어 입력 분포를 정규화하여 학습 안정화
            nn.LeakyReLU(0.2, inplace=True), # ReLU의 단점을 보완한 활성화 함수

            # 은닉층 1: 충분한 용량으로 복잡한 비선형 관계 학습
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2, inplace=True),

            # 은닉층 2: 학습된 특징을 점진적으로 요약하여 출력 차원으로 매핑 준비
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.LeakyReLU(0.2, inplace=True),

            # 출력층: 최종 잠재 벡터 차원으로 매핑. 활성화 함수 없음 (잠재 공간은 비제한적)
            nn.Linear(hidden_dim // 2, latent_dim)
        )

    def forward(self, z_a, z_b, alpha):
        # 입력으로 받은 3개의 텐서를 하나로 이어붙임
        # z_a, z_b: [Batch, 256], alpha: [Batch, 1]
        net_input = torch.cat([z_a, z_b, alpha], dim=1)
        return self.network(net_input)

In [3]:
from torch.utils.data import Dataset

class InterpolationDataset(Dataset):
    """
    사전 추출된 잠재 벡터 풀에서 무작위로 두 개의 벡터를 샘플링하고,
    무작위 alpha 값을 생성하여 학습 데이터를 구성하는 데이터셋.
    """
    def __init__(self, latent_vectors_path, num_samples):
        # 미리 추출하여 저장해 둔 실제 잠재 벡터(.pt 파일)를 불러옴 # GPU 텐서를 불러온 뒤, 즉시 CPU 메모리로 이동
        self.latent_vectors = torch.load(latent_vectors_path).to('cpu')
        self.num_latents = len(self.latent_vectors)
        self.num_samples = num_samples

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # 무작위로 두 잠재 벡터 선택
        idx1, idx2 = torch.randperm(self.num_latents)[:2]
        z_a = self.latent_vectors[idx1]
        z_b = self.latent_vectors[idx2]
        
        # 무작위로 alpha 값 생성
        alpha = torch.rand(1)
        
        return z_a, z_b, alpha

In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

# --- 1. 기본 설정 및 경로 ---
print("Step 1: 기본 설정 로드")
DEVICE = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
# VAE 체크포인트 경로
AE_CHECKPOINT_DIR = '/home/nas/data/YMG/superdeep_ae/checkpoints/'
ENCODER_PATH = os.path.join(AE_CHECKPOINT_DIR, 'encoder_superdeep_best.pth')
DECODER_PATH = os.path.join(AE_CHECKPOINT_DIR, 'decoder_superdeep_best.pth')
# 사전 추출된 잠재 벡터 경로
LATENT_VECTORS_PATH = '/home/nas/data/YMG/superdeep_ae/my_checkpoints/real_latent_vectors_20k.pt'
# 학습된 보간 네트워크를 저장할 경로
OUTPUT_DIR = '/home/nas/data/YMG/superdeep_ae/interpolation_network/'
os.makedirs(OUTPUT_DIR, exist_ok=True)
CHECKPOINT_PATH = os.path.join(OUTPUT_DIR, 'interpolation_net_best.pth')


# --- 하이퍼파라미터 ---
LATENT_DIM = 256
HIDDEN_DIM = 1024
BATCH_SIZE = 256
EPOCHS = 100
LR = 1e-4
SAMPLES_PER_EPOCH = 50000 # 한 에포크당 5만 개의 새로운 랜덤 쌍으로 학습


# --- 2. 모델 로드 및 설정 ---
print("Step 2: 모델 로드 및 설정")
# VAE 모델 (Encoder, Decoder) 로드 -> 가중치 동결
# (EncoderSuperDeep, DecoderSuperDeep 클래스 정의가 이 코드 위에 있어야 함)
encoder = EncoderSuperDeep().to(DEVICE)
decoder = DecoderSuperDeep().to(DEVICE)
encoder.load_state_dict(torch.load(ENCODER_PATH, map_location=DEVICE))
decoder.load_state_dict(torch.load(DECODER_PATH, map_location=DEVICE))
encoder.eval() # 평가 모드로 설정
decoder.eval() # 평가 모드로 설정
for param in encoder.parameters(): param.requires_grad = False # VAE 가중치 동결
for param in decoder.parameters(): param.requires_grad = False # VAE 가중치 동결
print("VAE 모델을 로드하고 가중치를 동결했습니다.")

# 보간 네트워크 모델 및 옵티마이저 초기화
interpolation_net = InterpolationNet(latent_dim=LATENT_DIM, hidden_dim=HIDDEN_DIM).to(DEVICE)
optimizer = optim.Adam(interpolation_net.parameters(), lr=LR)
print("보간 네트워크를 초기화했습니다.")


# --- 3. 데이터 로더 준비 ---
print("Step 3: 데이터 로더 준비")
train_dataset = InterpolationDataset(latent_vectors_path=LATENT_VECTORS_PATH, num_samples=SAMPLES_PER_EPOCH)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)


# --- 4. 학습 루프 ---
print("Step 4: 학습 시작")
best_loss = float('inf')
for epoch in range(EPOCHS):
    interpolation_net.train() # 학습 모드로 설정
    total_loss = 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    for z_a, z_b, alpha in pbar:
        z_a, z_b, alpha = z_a.to(DEVICE), z_b.to(DEVICE), alpha.to(DEVICE)
        
        # 순전파
        optimizer.zero_grad()
        z_interp = interpolation_net(z_a, z_b, alpha)
        
        # 재투영
        # 디코더/인코더는 4D 텐서(B, C, H, W)를 입력받으므로 모양 변경 필요
        z_interp_reshaped = z_interp.view(-1, 1, 16, 16)
        with torch.no_grad(): # VAE는 그래디언트 계산 불필요
            reconstructed_images = decoder(z_interp_reshaped)
            z_hat_interp_reshaped, _ = encoder(reconstructed_images)
        z_hat_interp = z_hat_interp_reshaped.view(-1, LATENT_DIM) # 다시 2D 텐서로 변경
        
        # 손실 계산
        loss = nn.functional.mse_loss(z_interp, z_hat_interp)
        
        # 역전파
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        pbar.set_postfix({"Re-projection Loss": f"{loss.item():.6f}"})
        
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} 완료 | Avg Loss: {avg_loss:.6f}")

    # 체크포인트 저장
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(interpolation_net.s vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv. z tate_dict(), CHECKPOINT_PATH)
        print(f"New best model found! Loss: {best_loss:.6f}. Checkpoint saved.")

print("\n학습이 모두 완료되었습니다.")

Step 1: 기본 설정 로드
Step 2: 모델 로드 및 설정


/tmp/ipykernel_3521935/1742408645.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder.load_state_dict(torch.load(ENCODER_PATH, map_location=DEVICE))
/tmp/ipykernel_

VAE 모델을 로드하고 가중치를 동결했습니다.
보간 네트워크를 초기화했습니다.
Step 3: 데이터 로더 준비
Step 4: 학습 시작


/tmp/ipykernel_3521935/604337460.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.latent_vectors = torch.load(latent_vectors_path).to('cpu')


Epoch 1/100:   0%|          | 0/196 [00:01<?, ?it/s]

...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd5b8274ee0>
Traceback (most recent call last):
  File "/home/a202192022/anaconda3/envs/GM-env/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/home/a202192022/anaconda3/envs/GM-env/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
    if w.is_alive():
  File "/home/a202192022/anaconda3/envs/GM-env/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd5b8274ee0>
Traceback (most recent call last):
  File "/home/a202192022/anaconda3/envs/GM-env/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/home/a202192022/anaconda3/en

Epoch 68 완료 | Avg Loss: 0.000336
New best model found! Loss: 0.000336. Checkpoint saved.


Epoch 69/100:   0%|          | 0/196 [00:00<?, ?it/s]

Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd5b8274ee0><function _MultiProcessingDataLoaderIter.__del__ at 0x7fd5b8274ee0>

Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/a202192022/anaconda3/envs/GM-env/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
  File "/home/a202192022/anaconda3/envs/GM-env/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
        self._shutdown_workers()self._shutdown_workers()

  File "/home/a202192022/anaconda3/envs/GM-env/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
  File "/home/a202192022/anaconda3/envs/GM-env/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
        if w.is_alive():if w.is_alive():

  File "/home/a202192022/anaconda3/envs/GM-env/lib/python3.9/multiprocessing/process.py", line 160, in is_aliv

KeyboardInterrupt: 

네, 물론입니다. 현재 2번 GPU 하나만 사용하고 있는데, 여러 개의 GPU를 모두 활용하여 학습 속도를 높이는 방법이 있습니다.

PyTorch에서 가장 간단하게 적용할 수 있는 방법은 이전에 VAE 학습 코드에서도 사용하셨던 \*\*`torch.nn.DataParallel`\*\*을 사용하는 것입니다.

-----

## **1. 가장 간단한 방법: `torch.nn.DataParallel`**

`DataParallel`은 모델을 감싸주기만 하면, 학습 시 데이터 배치(batch)를 여러 GPU에 자동으로 분산시켜 병렬로 처리해주는 편리한 기능입니다.

### **코드 수정 방법**

기존 학습 스크립트에서 **"2. 모델 로드 및 설정"** 부분만 아래와 같이 수정하면 됩니다.

#### **변경 전 (Before)**

```python
# --- 2. 모델 로드 및 설정 ---
print("Step 2: 모델 로드 및 설정")
# VAE 모델 (Encoder, Decoder) 로드 -> 가중치 동결
encoder = EncoderSuperDeep().to(DEVICE)
decoder = DecoderSuperDeep().to(DEVICE)
# ...
print("VAE 모델을 로드하고 가중치를 동결했습니다.")

# 보간 네트워크 모델 및 옵티마이저 초기화
interpolation_net = InterpolationNet(latent_dim=LATENT_DIM, hidden_dim=HIDDEN_DIM).to(DEVICE)
optimizer = optim.Adam(interpolation_net.parameters(), lr=LR)
print("보간 네트워크를 초기화했습니다.")
```

#### **변경 후 (After)**

```python
# --- 2. 모델 로드 및 설정 ---
print("Step 2: 모델 로드 및 설정")

# <<< 1. 사용할 모든 GPU ID를 리스트로 정의 >>>
# 주 GPU는 리스트의 첫 번째 GPU가 됩니다.
gpu_ids = [2, 1, 3, 0] # 사용하고 싶은 GPU 번호를 모두 적어주세요.
DEVICE = torch.device(f"cuda:{gpu_ids[0]}" if torch.cuda.is_available() else "cpu")
print(f"Main GPU: {DEVICE}, Parallel GPUs: {gpu_ids}")

# VAE 모델 로드
encoder = EncoderSuperDeep().to(DEVICE)
decoder = DecoderSuperDeep().to(DEVICE)
encoder.load_state_dict(torch.load(ENCODER_PATH, map_location=DEVICE))
decoder.load_state_dict(torch.load(DECODER_PATH, map_location=DEVICE))

# 보간 네트워크 모델 초기화
interpolation_net = InterpolationNet(latent_dim=LATENT_DIM, hidden_dim=HIDDEN_DIM).to(DEVICE)

# <<< 2. 모든 모델을 nn.DataParallel로 감싸기 >>>
encoder = nn.DataParallel(encoder, device_ids=gpu_ids)
decoder = nn.DataParallel(decoder, device_ids=gpu_ids)
interpolation_net = nn.DataParallel(interpolation_net, device_ids=gpu_ids)

# VAE 가중치 동결 (DataParallel로 감싼 후에도 동일하게 적용)
encoder.eval()
decoder.eval()
for param in encoder.parameters(): param.requires_grad = False
for param in decoder.parameters(): param.requires_grad = False
print("VAE 모델을 로드하고 DataParallel로 감쌌습니다.")

# 옵티마이저 설정
# DataParallel 사용 시, 실제 모델의 파라미터는 .module에 있습니다.
optimizer = optim.Adam(interpolation_net.module.parameters(), lr=LR)
print("보간 네트워크를 초기화하고 DataParallel로 감쌌습니다.")

# <<< 3. 체크포인트 저장/로드 시 .module 사용 (선택사항이지만 권장) >>>
# 체크포인트 저장 시
# torch.save(interpolation_net.module.state_dict(), CHECKPOINT_PATH)
# 체크포인트 로드 시
# interpolation_net.module.load_state_dict(torch.load(CHECKPOINT_PATH, map_location=DEVICE))
```

-----

## **2. 주의사항 및 한계**

1.  **배치 크기(Batch Size)**: 여러 GPU를 효과적으로 활용하려면 **배치 크기를 늘려주는 것이 좋습니다.** 예를 들어, GPU 4개를 사용한다면 기존 `BATCH_SIZE`를 2\~4배로 늘려야 눈에 띄는 속도 향상을 기대할 수 있습니다. (`BATCH_SIZE = 512` 또는 `1024` 등)

2.  **GPU-0 병목 현상**: `DataParallel`은 구조적으로 리스트의 첫 번째 GPU(여기서는 `cuda:2`)에 데이터를 모아 손실을 계산하는 등 더 많은 작업을 시킵니다. 이 때문에 GPU 사용량이 불균등할 수 있으며, 이것이 속도 향상을 제한하는 요인이 되기도 합니다.

-----

## **3. 더 빠르고 전문적인 방법: `DistributedDataParallel` (DDP)**

`DataParallel`의 병목 현상을 해결하고, 거의 선형적인 속도 향상을 원할 때는 \*\*`DistributedDataParallel`(DDP)\*\*를 사용하는 것이 업계 표준입니다. DDP는 각 GPU가 독립적인 프로세스로 동작하여 훨씬 효율적으로 통신합니다.

다만, DDP는 스크립트를 실행하는 방식부터 코드 구조까지 변경해야 할 부분이 많아 `DataParallel`보다 구현이 복잡합니다.

**결론적으로,** 가장 빠르고 간단하게 다중 GPU 학습을 적용해보시려면, 위 코드처럼 \*\*`DataParallel`\*\*을 사용하고 **배치 크기를 늘리는 것**을 추천합니다. 🚀